In [25]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-1x1x1/"

In [26]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [27]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# Read wfndata
wfndata = wfn2py(dirname + "WFN_co.h5")
wfnqdata = wfn2py(dirname + "WFNq.h5")

# Read rho
rho = inp.read_rho(dirname + "RHO")

# Read vxc 
vxc = inp.read_vxc(dirname + "vxc.dat") 

In [28]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 1/1 [00:00<00:00, 6932.73it/s]


In [29]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2


        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv0 = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)
        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv0, indices_gspace_sorted)

        epsilon.l_epsinv[i_q] = epsinv

        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))


        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")


    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 1/1 [00:00<00:00, 346.72it/s]

Standard deviation exceeded 1e-16 tolerance: 1.9894890166455426e-15, for i_q:0


In [30]:
from kernel import KernelMtxEl

q0 = [0.001, 0.001, 0.001]
l_qpts = np.array(epsinp.qpts)
l_qpts[0] *= 0
qpts = QPoints.from_cryst(wfndata.kpts.recilat, None, *l_qpts)


In [31]:
kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    l_epsmats=epsilon.l_epsinv,
    q0=q0,
    parallel=False,
    num_bands_val=3,
    num_bands_con=2,
)

In [32]:
data = kernelclass.kernel_mtxel()

exc = data["exc"]
exc = exc[0, 0, 0, 0, :, :]
print(f"The exchange kernel is {exc}.\n")

head = data["head"]
head = head[0, 0, 0, 0, :, :]
print(f"The head kernel is {head}.\n")

wings = data["wings"]
wings = wings[0, 0, 0, 0, :, :]
print(f"The wings kernel is {wings}.\n")

body = data["body"]
body = body[0, 0, 0, 0, :, :]
print(f"The body kernel is {body}.\n")

The exchange kernel is [[ 3.93264247e-01-2.83530394e-18j -5.45739046e-05+9.19039038e-04j
  -1.87397841e-03+1.74508940e-03j]
 [-5.45739046e-05-9.19039038e-04j  4.84342648e-01-1.60716059e-18j
   1.04822686e-03+9.99524167e-04j]
 [-1.87397841e-03-1.74508940e-03j  1.04822686e-03-9.99524167e-04j
   4.87372711e-01-2.77555756e-17j]].

The head kernel is [[ 1.00000000e+00-1.89016773e-19j  6.42680544e-17-8.52846219e-18j
  -6.09175871e-17-2.43670348e-17j]
 [ 6.42680544e-17+9.74681393e-18j  1.00000000e+00-6.75028197e-20j
  -4.62973662e-17-1.55949023e-16j]
 [-6.09175871e-17+2.43670348e-17j -4.62973662e-17+1.46202209e-16j
   1.00000000e+00-7.82407188e-19j]].

The wings kernel is [[0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j]].

The body kernel is [[-2.08308297e-02+1.04335341e-08j -7.43919575e-05+1.24928541e-03j
  -4.99875688e-04+4.65361363e-04j]
 [-7.43848003e-05-1.24928604e-03j  3.22741789e-01+4.24464628e-09j
   7.06323975e-04+6.73437186e-04j]
 [-4.99874415e-04-4.65362096e-